In [3]:
## cuda device 설정
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"]= "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)

Device: cuda
Count of using GPUs: 1
Current cuda device: 0


# 평가 데이터셋 구축(ver.kss-zeroth)

In [2]:
from setproctitle import setproctitle
setproctitle("wav2vec metric_check")

from datasets import Dataset
from tqdm import tqdm
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

import numpy as np
import soundfile as sf
import re
import os
import torch

dataset_path = "/wav2vec2/s-kr/fine-tune/dataset"
model_path = os.path.join(dataset_path, "results/zeroth_only")
processor_path = os.path.join(dataset_path, 'results/zeroth_only')

processor = Wav2Vec2Processor.from_pretrained(processor_path)

input_values_list = list()
input_length_list = list()
labels_list = list()

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def _read_txt_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        
    return text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
zeroth_path = os.path.join(dataset_path, "zeroth/test_data")
for folder in tqdm(sorted(os.listdir(zeroth_path))):
    folder_path = os.path.join(zeroth_path, folder)
    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".flac"):
            flac_path = os.path.join(folder_path, file)
            audio, _ = sf.read(flac_path)
            if audio.ndim > 1:
                audio = np.delete(audio, 1, axis=1)
                audio = audio.reshape(-1)
            
            input_value = processor(audio, sampling_rate=16000).input_values[0]
            input_values_list.append(input_value)
            
            input_length_list.append(len(input_value))
            
            txt_path = flac_path.replace(".flac", ".txt")
            text = _read_txt_file(txt_path)
            text = re.sub(chars_to_ignore_regex, '', text)                             
            with processor.as_target_processor():
                labels_list.append(processor(text).input_ids)

In [ ]:
print(len(input_values_list))
print(input_values_list[0], input_values_list[0].shape)
print(input_length_list[0])
print(labels_list[0])

In [ ]:
test_df = pd.DataFrame({'input_values': input_values_list, 'input_length': input_length_list, 'labels': labels_list})
test_df.head()

In [ ]:
test_timit = Dataset.from_pandas(test_df)
test_timit

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

# 평가 진행(ver.kss-zeroth)

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    model_path
).to('cuda')

mode.config

In [ ]:
results = test_timit.map(map_to_result, batch_size=2, remove_columns=test_timit.column_names)

print(results['pred_str'][:5])
print("-" * 20)
print(results['text'][:5])

In [ ]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results["pred_str"], references=results["text"])))

# 평가 데이터셋 구축 및 평가 진행(ver. naver clova)

In [1]:
from setproctitle import setproctitle
setproctitle("Clova metric_check")

from tqdm import tqdm

import os
import Levenshtein as Lev


def cal_cer(input_text: str, output_text: str) -> float:
    input_text = input_text.replace(' ', '')
    output_text = output_text.replace(' ', '')
    
    dist = Lev.distance(output_text, input_text)
    length = len(input_text)
    
    return dist / length


def _read_txt_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        
    return text

In [7]:
# base_path = "/wav2vec2/s-kr/fine-tune/dataset/zeroth/test_data"
# naver_data_path = os.path.join(base_path, "137/out_file")
naver_data_path = "/wav2vec2/s-kr/fine-tune/dataset/sbs"

naver_text_list = list()
answer_list = list()
for naver_data in tqdm(sorted(os.listdir(naver_data_path))):
    # if naver_data.endswith(".txt"):
    if (naver_data.endswith(".txt")) & ("nc" in naver_data):
        naver_text_path = os.path.join(naver_data_path, naver_data)
        naver_text_list.append(_read_txt_file(naver_text_path))
        
# for data in tqdm(sorted(os.listdir(os.path.join(base_path, "137")))):
for data in tqdm(sorted(os.listdir(naver_data_path))):
    # if data.endswith(".txt"):
    if (data.endswith(".txt")) & ("nc" not in data):
        # text_path = os.path.join(base_path, "137", data)
        text_path = os.path.join(naver_data_path, data)
        answer_list.append(_read_txt_file(text_path))

# answer_list = answer_list[1:]

print(len(naver_text_list))
print(len(answer_list))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 10722.73it/s]

8
8


In [8]:
print(naver_text_list[-1])
print(answer_list[-1])

여러분 안녕하십니까 어제 치러진 지방선거는 국민의힘의 압승 민주당의 완패였습니다 전국 광역단체장 열일곱곳 가운데 여당인 국민의힘이 열두곳을 가져가며 지난 대선에 이어 지방 정부에서도 정권 교체가 이뤄졌습니다 반면 사년 전 지방선거 때는 광역단체장 열네곳을 휩쓸었던 민주당은 이번에는 호남 지역과 제주 그리고 경기 모두 다섯곳에서만 자리를 지켰습니다 지방선거와 함께 치러진 국회의원 보궐선거 결과까지 먼저 화강윤 기자가 정리해 드리겠습니다 앞으로 사년 동안 각 지방 정부의 행정을 책임지게 될 광역 시도지사 당선인은 민주당 소속이 다섯명 국민의힘 소속이 열두명입니다 사년 전과 비교하면 민주당 광역단체장은 열네명에서 다섯명으로 확 줄었고 국민의힘은 당시 무소속으로 출마한 원희룡 제주지사를 포함해 세명에서 열두명으로 쑥 늘었습니다
여러분 안녕하십니까 어제 치러진 지방선거는 국민의힘의 압승 민주당의 완패였습니다 전국 광역단체장 열일곱곳 가운데 여당인 국민의힘이 열두곳을 가져가며 지난 대선에 이어 지방정부에서도 정권 교체가 이루어졌습니다 아 반면 사년 전 지방선거 때는 광역단체장 열네곳을 휩쓸었던 민주당은 이번엔 호남 지역과 제주 그리고 경기 모두 다섯곳에서만 자리를 지켰습니다 자 지방선거와 함께 치러진 국회의원 보궐선거 결과까지 먼저 화강윤 기자가 정리해드리겠습니다 앞으로 사년 동안 각 지방정부의 행정을 책임지게 될 광역시도지사 당선인은 민주당 소속이 다섯명 국민의힘 소속이 열두명입니다 사년 전과 비교하면 민주당 광역단체장은 열네명에서 다섯명으로 확 줄었고 국민의힘은 당시 무소속으로 출마한 원희룡 제주지사를 포함해 세명에서 열두명으로 쑥 늘었습니다


In [10]:
for naver_data, answer_data in zip(naver_text_list, answer_list):
    print(naver_data, ":::::", answer_data)
    break

울타리가 돼야 하는 가정이 무너지고 있는 건 아닌지 걱정되는 소식 하나 더 전해드립니다 오늘 오전 한 사십대 가장이 부인과 두 자녀를 둔기로 무참히 살해하고 스스로 목숨을 끊었습니다 보도에 화강윤 기자입니다 오늘 오전 아홉시 오분쯤 경기도 광주의 한 아파트에서 마흔여덞살 최 모 씨의 부인과 아들 딸 세명이 숨진 채 발견됐습니다 자신이 둔기로 부인과 아이들을 때렸는데 숨을 안 쉰다며 신고를 했던 최 씨도 아파트 아래 인도에서 숨진 채 발견됐습니다 추락 당시의 충격으로 아파트 난간이 이렇게 형편없이 부서져 있습니다 아버지 최 씨는 자신의 범행을 신고한 뒤 창문을 통해 스스로 뛰어내린 것으로 추정됩니다 아버지 최 씨는 우울증과 불면증 등으로 지난해 구월부터 치료를 받아온 것으로 확인됐습니다 ::::: 울타리가 돼야하는 가정이 무너지고 있는 건 아닌지 걱정되는 소식 하나 더 전해드립니다 오늘 오전 한 사십대 가장이 부인과 두 자녀를 둔기로 무참히 살해하고 스스로 목숨을 끊었습니다 보도에 화강윤 기자입니다 오늘 오전 아홉시 오분쯤 경기도 광주의 한 아파트에서 마흔 여덞살 최 모 씨의 부인과 아들 딸 세명이 숨진 채 발견됐습니다 자신이 둔기로 부인과 아이들을 때렸는데 숨을 안 쉰다며 신고를 했던 최 씨도 아파트 아래 인도에서 숨진 채 발견됐습니다 추락 당시 충격으로 아파트 난간이 이렇게 형편없이 부서져 있습니다 아버지 최 씨는 자신의 범행을 신고한 뒤 창문을 통해 스스로 뛰어내린 것으로 추정됩니다 아버지 최 씨는 우울증과 불면증 등으로 지난해 구월부터 치료를 받아 온 것으로 확인됐습니다



In [11]:
total_cer = 0.0
for naver_data, answer_data in zip(naver_text_list, answer_list):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer

result_cer = total_cer / len(answer_list)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.019
Test ACC: 98.124


# 성능 평가

In [1]:
from setproctitle import setproctitle
setproctitle("stt metric_check")

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from tqdm import tqdm

import os
import Levenshtein as Lev
import soundfile as sf
import librosa
import torch

os.environ["CUDA_VISIBLE_DEVICES"]= "3"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)

base_path = "/wav2vec2/s-kr/fine-tune/dataset"

# test_data_path = os.path.join(base_path, "zeroth/test_data/137")
# test_data_path = os.path.join(base_path, "sbs")
test_data_path = os.path.join(base_path, "zeroth/test_data")

# infer_path = os.path.join(base_path, "results/checkpoint-7000")
infer_path = os.path.join(base_path, "results/checkpoint-50000")
# infer_path = os.path.join(base_path, "results/kss_zeroth")
# infer_path = os.path.join(base_path, "results/zeroth_only")
# infer_path = os.path.join(base_path, "results/kss_zeroth")
# infer_path = "kresnik/wav2vec2-large-xlsr-korean"

def cal_cer(input_text: str, output_text: str) -> float:
    input_text = input_text.replace(' ', '')
    output_text = output_text.replace(' ', '')
    
    dist = Lev.distance(output_text, input_text)
    length = len(input_text)
    
    return dist / length


def read_text(file_path):
    with open(file_path, 'r') as f:
        text = f.read()
    
    return text

Device: cuda
Count of using GPUs: 1


In [2]:
# 정답 데이터 준비
label_data = list()
for test_data in tqdm(sorted(os.listdir(test_data_path))):
    data_path = os.path.join(test_data_path, test_data)
    for data in sorted(os.listdir(data_path)):
        if data.endswith(".txt"):
            txt_path = os.path.join(data_path, data)
        # if test_data.endswith(".txt") and ("nc" not in test_data):
        #     txt_path = os.path.join(test_data_path, test_data)
            
            text = read_text(txt_path)
            if text != '':
                label_data.append(text)

# label_data = label_data[1:]

print(len(label_data))
print(label_data[:2])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 198.98it/s]

457
['지난해 삼 월 김 전 장관의 동료인 장동련 홍익대 교수가 민간 자문단장으로 위촉되면서 본격적인 공모와 개발 작업에 들어갔다', '그 바람에 나의 몸도 겹쳐 쓰러지며 한창 피어난 노란 동백꽃 속으로 폭 파묻혀 버렸다']


In [3]:
## 추론 모델 준비
# 1. wav2vec2.0
processor = Wav2Vec2Processor.from_pretrained(infer_path)

model = Wav2Vec2ForCTC.from_pretrained(infer_path).to('cuda')

# 2.deepspeech2

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 추론 테스트 데이터 준비
input_values = list()

for file in tqdm(sorted(os.listdir(test_data_path))):
    file_path = os.path.join(test_data_path, file)
    for flac_file in sorted(os.listdir(file_path)):
        if flac_file.endswith(".flac"):
        # if file.endswith(".wav"):
            flac_path = os.path.join(file_path, flac_file)
            # flac_path = os.path.join(test_data_path, file)
            audio, _ = sf.read(flac_path)
            # audio, _ = librosa.load(flac_path, sr=16000)
            if audio.ndim > 1:                              
                audio = np.delete(audio, 1, axis=1)
                audio = audio.reshape(-1)
            input_values.append(processor(audio, sampling_rate=16000).input_values[0])
        
print(len(input_values))        
print(input_values[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.56it/s]

457
[ 0.00082447  0.00164228  0.00218748 ... -0.00108373  0.00164228
 -0.00108373]


In [5]:
# 모델 추론 동작 및 저장
results = list()
for value in tqdm(input_values):
    with torch.no_grad():
        value = torch.tensor(value, device="cuda").unsqueeze(0)
        logits = model(value).logits

    pred_ids = torch.argmax(logits, dim=-1)
    result = processor.batch_decode(pred_ids)[0]
    result = result.replace('[PAD]', '')
    results.append(result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 457/457 [00:23<00:00, 19.40it/s]


In [6]:
for result in results:
    print(result)

지한에 삼 월 증전장관에 동료인 장동이년 홍익대 교수가 민간 자문 단장으로 미쪽되면서 동격적인 공모아 개발작업에 들어갔다
음 그 만함에 나에 몸도 겹쳐스러주면 한창취형한 노란 동대 버속으로 꼭 한무쳐버렸다
현재 백화전과 영화관등는 분해 영업하고 있고 혜마 파큼이 아파트딩에 공산은 이천시 한 년 완공을 목표로 지낸겨 왔다
또한 박전된 동명측은 앞으로 경찰할 수사과 관련해 수사 대용이 대한 입장꾼 아니라 변호인단에 활동 상황등도 수상하게 공기한다는 강치이가
한전에 커피가 소기 자에게 전해지게 위해선 여러 차례 공정을 거쳐야 한다
다른 지역에서는 귀것이 아무리 넓다 하더라도 프야 라이몽기를 발견하기가 어려울 것이니까
물론 실대 자녀가 잘못된 결론을 내리거나 신지어 당신과 비슷한 실수를 전질렀을 게 자신을 정당화하지는 않을까 영려가 될 수도 있습니까
만약 바은 애들 통령이 정말로 손을 내미면 안 철소에 표아 국민에다는 어떤 선택을 할까요
대북은 전기는 기엽이 사용 가정용 전기에 류진재 적용하는 국가 거의 없어 많이 쓴다고 돈 전원 얘기하는 횡상품이 어딨나
더 서은 이름이 밝혀지지 않은 이 노인이 이 년 전 피부함물 지난받은 일 자신의 양신과 실왔다고 전했다
생명역이 강한 이 미생물들은 바닷물에 서추출한물 질과 끈끈한 전액을 섞아서 접밥질을 만든다는 바위같 치 생긴 집에 층층히 바릅니다
떼때로는 반이 풀바했을 치는 소리를 들으면서 그 소리 많은 대조적으로 크고도 찢어지는 듯이 나이카락 한 싼띄물 때세일 우음소리가 물퉁 근처에서 들려 오는 것에 귀를 기울리건 하셨습니까
육십 사 년 초임 검사로 인관한 김전실장은 십 년후윈 칠십 사 년 사 월 중앙 전보 부장 법률보자관으로 확연됐고 같은 애 구 월 대공 수삭 육장이겠다
하지만 이 같은 실수는 경쟁사를 이시구해 줄석으로 진행견일를 마트 할인행사에 강면을 들렁냈다는 점에서 소비전들에 시선이 곡지 않다
긴범서 쿠팡대표는 지난에 십 일 월 삼 간담에 를 열고 계속 일 년 강화를 위에 이천 십 육 년 한만명 이천 십 칠 년 일만 다

In [7]:
# cer 측정
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.199
Test ACC: 80.060


In [6]:
# cer 측정 - kposn-checkpoint-44000 / sbs
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.217
Test ACC: 78.317


In [6]:
# cer 측정 - kposn-checkpoint-44000 / zeroth 457
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.211
Test ACC: 78.868


In [6]:
# cer 측정 - kposn-checkpoint-15000 / zeroth 457
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.238
Test ACC: 76.157


In [8]:
# cer 측정 - kposn-checkpoint-15000 / sbs
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.249
Test ACC: 75.140


In [12]:
# cer 측정 - kposn-checkpoint-41800
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.286
Test ACC: 71.387


In [7]:
# cer 측정 - checkpoint-8400
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.080
Test ACC: 91.972


In [6]:
# cer 측정 - checkpoint-7000
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.096
Test ACC: 90.431


In [22]:
# cer 측정 - hugging face
total_cer = 0.0
for naver_data, answer_data in zip(results, label_data):
    cer = cal_cer(answer_data, naver_data)
    total_cer += cer
    
result_cer = total_cer / len(results)
    
print("Test CER: {:.3f}".format(result_cer))
print("Test ACC: {:.3f}".format((1-result_cer) * 100))

Test CER: 0.017
Test ACC: 98.294


# 추론 테스트

In [15]:
from setproctitle import setproctitle
setproctitle("wav2vec infer test")

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
import os
import torch

dataset_path = "/wav2vec2/s-kr/fine-tune/dataset"
infer_path = os.path.join(dataset_path, 'results/zeroth_only')
# infer_path = os.path.join(dataset_path, 'results/kss_only')
# infer_path = os.path.join(dataset_path, 'results/kss_zeroth')
# infer_path = os.path.join(base_path, "results/checkpoint-3600")
# infer_path = "kresnik/wav2vec2-large-xlsr-korean"

processor = Wav2Vec2Processor.from_pretrained(infer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)

Device: cuda
Count of using GPUs: 1
Current cuda device: 0


In [17]:
model = Wav2Vec2ForCTC.from_pretrained(
    infer_path).to('cuda')

In [19]:
from tqdm import tqdm
import soundfile as sf
import numpy as np
import librosa

input_values = list()
# test_data_path = os.path.join(dataset_path, "zeroth/test_data/137")
test_data_path = os.path.join(dataset_path, "sbs")

for file in tqdm(sorted(os.listdir(test_data_path))):
    # if file.endswith(".flac"):
    if file.endswith(".wav"):
        flac_path = os.path.join(test_data_path, file)
        # audio, _ = sf.read(flac_path)
        audio, _ = librosa.load(flac_path, sr=16000)
        if audio.ndim > 1:                              
            audio = np.delete(audio, 1, axis=1)
            audio = audio.reshape(-1)
        input_values.append(processor(audio, sampling_rate=16000).input_values[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:19<00:00,  1.31it/s]


In [20]:
for values in input_values:
    print(values.shape)

(851737,)
(848001,)
(985803,)
(1475024,)
(909488,)
(1145474,)
(1099703,)
(988640,)


In [21]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(input_values)-1)

ipd.Audio(data=np.asarray(input_values[rand_int]), rate=16000)  # 상대적으로 발성이 느리고 발음이 불문명

In [22]:
results = list()
for value in tqdm(input_values):
    with torch.no_grad():
        value = torch.tensor(value, device="cuda").unsqueeze(0)
        logits = model(value).logits

    pred_ids = torch.argmax(logits, dim=-1)
    result = processor.batch_decode(pred_ids)[0]
    result = result.replace('[PAD]', '')
    results.append(result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:04<00:00,  1.82it/s]


In [23]:
for result in results:
    print(result)

울타리가 되야하는 가정이 문어지고 있는 건 아년제 걱정되행 소식 사나더 전해 들입 오으로전 한 사십 대 가장이 부인과 두 자녀를 둔기로 무참이 사례하고 스스로 목씀을 그었다보도에 화강윤기점오늘 오전 아홉이 오은 쯤 경기도 광주에 한 아파트에서 마흔 여덟 살 최모씨에 부인과 아들 달 세 명이 순진체 발견됐습자신이 둔기로 부인과 아이들을 때렸는데 숨을 안쉬인다며 신고를 했던 최씨도 아파트 알에 인도에서 숨진체 발견됐출락당실 충격으로 아파트 안간이 이렇게 형편없이 부서될습니다 아부지 최씨는 자신의 보음액을 신고한 뒤 창문을 통해 스스로 뛰어내린 것으로 추정됩니아부지 최씨는 우울증과 불면 증 등으로 지난해 구월부터 치료를 받다온 것으로 확인됐습니다
저 미국이 이렇게 나오면 우리나라도 영향을 받들 수밖에 없습다 당장 한국은행이 지난달에 이어서 두 달 연속 금리를 온리 가능성이 큼니다 아인상 복도 미국처럼 영 쩜 오 퍼센트 포인트록 크게 잡아야 한단 이야기까지나하고 있에내용은 전연남기자가 집어왔습우리나라 물까도 미국못지 않게 심각합 사 월 소비자물과 상승률이 십 삼 년 만에 최곳치인 사 쩜 팔 퍼센트를 기록했여기의 앞으로 물까가 오릴 거라는 심리 즉 기대 임플레이션이 뒤는 것도 문점서비스요금이나 임금을 올려받아야 겠다는 생각이 퍼지면서 물가 상승을 더 부축이기 때문입이창용 한국은행총제는 이런 기대십 리를 미리 누르기 위해서라도 금리 인상이 필요하다고 밝켜왔습니다
그러나가 자자들면서 오랜 만에 어린 이나을 다왔던 오 월 오일입다 사아졌던 어린이날 행사가 삼 년 만에 다시 열렸고 아이들이 조와하는 노리공원엔 오랜만에 긴줄이 늘어섰니다  차도 많인일리고 가는 군마다 사람이 찬 많았지만 그래도 아이들의 운눈 얼구리 어느 때보다 반가왔던 하요였래로 백 주년을 마진 어린이날 풍경 먼저 소한후기자가 전해들이노리공원은 오 점부쳐 말그대로 인산 인했노리기고 탑승을 기더리는 줄은 끝지 안볼정도지만 오올랜 기다린 끝해 올랐한 론러코스터에서 자리담을 만끼합오랜마에 나온 야외 수용장에서 아빠엄

# GEC 데이터셋 만들기
- 숙주쌤이 사용할 zeroth 데이터셋 만들기

In [3]:
from setproctitle import setproctitle
setproctitle("GEC dataset")

from glob import glob
from tqdm import tqdm
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

import os
import torch
import soundfile as sf
import numpy as np
import pandas as pd


os.environ["CUDA_VISIBLE_DEVICES"]= "3"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)

dataset_path = "/wav2vec2/s-kr/fine-tune/dataset"
# train_data_path = os.path.join(dataset_path, "zeroth/train_data")
# test_data_path = os.path.join(dataset_path, "zeroth/test_data")
kspon_path = os.path.join(dataset_path, "KsponSpeech")

# infer_model = "kresnik/wav2vec2-large-xlsr-korean"
infer_model = "/wav2vec2/s-kr/fine-tune/dataset/results/kspon_240h_only"

processor = Wav2Vec2Processor.from_pretrained(infer_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Count of using GPUs: 1


In [4]:
# train_flacs = sorted(glob(os.path.join(train_data_path, '**', '*.flac'), recursive=True))
# test_flacs = sorted(glob(os.path.join(test_data_path, '**', '*.flac'), recursive=True))
kspon_wavs = sorted(glob(os.path.join(kspon_path, '**', '*.wav'), recursive=True))
kspon_wavs = kspon_wavs[:1000]
# print(len(train_flacs))
# print(len(test_flacs))

In [5]:
import librosa

input_values = list()
text_values = list()

# for flac in tqdm(train_flacs):
#     audio, _ = sf.read(flac)
#     if audio.ndim > 1:
#         audio = np.delete(audio, 1, axis=1)
#         audio = audio.reshape(-1)
#     input_values.append(processor(audio, sampling_rate=16000).input_values[0])
    
#     with open(flac.replace(".flac", ".txt"), 'r') as f:
#         text = f.read()
#     text_values.append(text)
# print("Trainset Complete!")

# for flac in tqdm(test_flacs):
#     audio, _ = sf.read(flac)
#     if audio.ndim > 1:
#         audio = np.delete(audio, 1, axis=1)
#         audio = audio.reshape(-1)
#     input_values.append(processor(audio, sampling_rate=16000).input_values[0])
    
#     with open(flac.replace(".flac", ".txt"), 'r') as f:
#         text = f.read()
#     text_values.append(text)
# print("Testset Complete!")

for wavs in tqdm(kspon_wavs):
    audio, _ = sf.read(wavs)
    if audio.ndim > 1:
        audio = np.delete(audio, 1, axis=1)
        audio = audio.reshape(-1)
    input_values.append(processor(audio, sampling_rate=16000).input_values[0])
    
    with open(wavs.replace(".wav", ".txt"), 'r', encoding='cp949') as f:
        text = f.read()
    text_values.append(text)
print("Testset Complete!")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 532.59it/s]

Testset Complete!


In [6]:
model = Wav2Vec2ForCTC.from_pretrained(infer_model).to('cuda')

In [7]:
import gc

results = list()
for value in tqdm(input_values):
    with torch.no_grad():
        value = torch.tensor(value, device="cuda").unsqueeze(0)
        logits = model(value).logits

    pred_ids = torch.argmax(logits, dim=-1)
    result = processor.batch_decode(pred_ids)[0].replace('[PAD]', '')
    results.append(result)
else:
    del results
    gc.collection()
    
print(len(results))
print(len(text_values))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:54<00:00, 18.22it/s]

1000
1000


In [8]:
print(results[:10])
print("="*60)
print(text_values[:10])

['아 뭔 소리야 그건 또', '나는 학습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로 하나 정도쯤은 있어야 되거든  물 들어가고', '그래서 지오랑 계산 올라가서 막 위에 운동하는 기구 있대요 그서 그걸로 운동할려고요', '그게 영 점 일 프로 가정의 아이들과 가정의 모습이야', '그 친해하는 판산님께라는 법 법 관련 드라마 알고 있어', '그래', '나도 몰라 나 그 시리즈 하나도 안 봤음 어', '아 내일 나 알바하구나', '아 근데 한 시에 닫는 게 좀 아쉽긴 한데 거기 진짜 괜찮은데', '맞아 시간 안에 풀고 약간 이것부터 풀고 요런식으로 풀어보렴 이런 거']
['아/ 몬 소리야, 그건 또. b/\n', '나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고.\n', 'b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. b/ 그서 그걸로 운동 할려구요. b/ n/\n', 'o/ b/ 그게 (0.1프로)/(영 점 일 프로) 가정의 아이들과 가정의 모습이야? b/\n', '그/ 친애하는 판사님께라는 법+ 법 관련 드라마 알고 있어?\n', '그래\n', 'o/ 나도 몰라. 나 그/ (3G)/(쓰리 쥐)* 하나도 안 봤음. 어.\n', '아/ 내일 나 알바하구나.\n', 'n/ 아/ 근데 (1시)/(한 시)에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 b/\n', '맞아. 시간 안에 풀고 약간 이것부터 풀고 요런 식으로 풀어보렴. 이런 거 b/\n']


In [9]:
gec_dataset_path = os.path.join(dataset_path, "gec_data1.tsv")

gec_df = pd.DataFrame({"form":results, "corrected_form":text_values})
gec_df.head()

,form,corrected_form
0,아 뭔 소리야 그건 또,"아/ 몬 소리야, 그건 또. b/\n"
1,나는 학습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로...,나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악...
2,그래서 지오랑 계산 올라가서 막 위에 운동하는 기구 있대요 그서 그걸로 운동할려고요,b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. ...
3,그게 영 점 일 프로 가정의 아이들과 가정의 모습이야,o/ b/ 그게 (0.1프로)/(영 점 일 프로) 가정의 아이들과 가정의 모습이야?...
4,그 친해하는 판산님께라는 법 법 관련 드라마 알고 있어,그/ 친애하는 판사님께라는 법+ 법 관련 드라마 알고 있어?\n


In [13]:
from kspon_preprocess import special_filter, bracket_filter

gec_df['corrected_form'] = gec_df['corrected_form'].apply(lambda x: special_filter(bracket_filter(x)))
gec_df.head()

,form,corrected_form
0,아 뭔 소리야 그건 또,"아 몬 소리야, 그건 또."
1,나는 학습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로...,나는 악습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로...
2,그래서 지오랑 계산 올라가서 막 위에 운동하는 기구 있대요 그서 그걸로 운동할려고요,그래서 지호랑 계단 올라와서 막 위에 운동하는 기구 있대요. 그서 그걸로 운동 할려구요.
3,그게 영 점 일 프로 가정의 아이들과 가정의 모습이야,그게 영 점 일 프로 가정의 아이들과 가정의 모습이야?
4,그 친해하는 판산님께라는 법 법 관련 드라마 알고 있어,그 친애하는 판사님께라는 법 법 관련 드라마 알고 있어?


In [17]:
gec_df.to_csv(gec_dataset_path, sep="\t")

In [2]:
'''
GEC 데이터셋 동작 최종 코드
해당 셀과 바로 아래 셀, 총 두 개의 셀만 동작하면 끝.
'''
from setproctitle import setproctitle
setproctitle("GEC dataset")

from glob import glob
from tqdm import tqdm
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from kspon_preprocess import special_filter, bracket_filter

import os
import torch
import soundfile as sf
import numpy as np
import pandas as pd
import gc


os.environ["CUDA_VISIBLE_DEVICES"]= "3"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)

dataset_path = "/wav2vec2/s-kr/fine-tune/dataset"
kspon_path = os.path.join(dataset_path, "KsponSpeech")

infer_model = "/wav2vec2/s-kr/fine-tune/dataset/results/kspon_240h_only"

processor = Wav2Vec2Processor.from_pretrained(infer_model)
model = Wav2Vec2ForCTC.from_pretrained(infer_model).to('cuda')

kspon_wavs = sorted(glob(os.path.join(kspon_path, '**', '*.wav'), recursive=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Count of using GPUs: 1


In [4]:
i = 0
for idx in tqdm(range(0, len(kspon_wavs), 10000), desc='total wavs processing...'):
    if idx + 10000 < len(kspon_wavs):
        wavs = kspon_wavs[idx:idx+10000]
    else:
        wavs = kspon_wavs[idx:]
    
    input_values = list()
    text_values = list()
    # for wav in tqdm(wavs, desc='wav-text file processing'):
    for wav in wavs:
        audio, _ = sf.read(wav)
        if audio.ndim > 1:
            audio = np.delete(audio, 1, axis=1)
            audio = audio.reshape(-1)
        input_values.append(processor(audio, sampling_rate=16000).input_values[0])

        with open(wav.replace(".wav", ".txt"), 'r', encoding='cp949') as f:
            text = f.read()
        text_values.append(text)   
        
    results = list()
    # for value in tqdm(input_values, desc='inference processing...'):
    for value in input_values:
        with torch.no_grad():
            value = torch.tensor(value, device="cuda").unsqueeze(0)
            logits = model(value).logits

        pred_ids = torch.argmax(logits, dim=-1)
        result = processor.batch_decode(pred_ids)[0].replace('[PAD]', '')
        results.append(result) 
    
    gec_dataset_path = os.path.join(dataset_path, f"gec_dataset/gec_data{i}.tsv")
    gec_df = pd.DataFrame({"form":results, "corrected_form":text_values})
    gec_df['corrected_form'] = gec_df['corrected_form'].apply(lambda x: special_filter(bracket_filter(x)))
    gec_df.to_csv(gec_dataset_path, sep="\t")
    print("Complete making tsv file: ", gec_dataset_path)
    i += 1
    
    del input_values, wavs, results, text_values
    gc.collect()

total wavs processing...:   2%|█▌                                                                                           | 1/62 [08:22<8:30:39, 502.29s/it]

Complete making tsv file:  /wav2vec2/s-kr/fine-tune/dataset/gec_dataset/gec_data0.tsv


total wavs processing...:   3%|███                                                                                          | 2/62 [16:48<8:24:49, 504.82s/it]

Complete making tsv file:  /wav2vec2/s-kr/fine-tune/dataset/gec_dataset/gec_data1.tsv


total wavs processing...:   5%|████▌                                                                                        | 3/62 [25:12<8:15:51, 504.26s/it]

...

Complete making tsv file:  /wav2vec2/s-kr/fine-tune/dataset/gec_dataset/gec_data58.tsv


total wavs processing...:  97%|█████████████████████████████████████████████████████████████████████████████████████████   | 60/62 [7:48:53<17:54, 537.09s/it]

Complete making tsv file:  /wav2vec2/s-kr/fine-tune/dataset/gec_dataset/gec_data59.tsv
Complete making tsv file:  /wav2vec2/s-kr/fine-tune/dataset/gec_dataset/gec_data60.tsv


total wavs processing...: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [8:00:41<00:00, 465.18s/it]

Complete making tsv file:  /wav2vec2/s-kr/fine-tune/dataset/gec_dataset/gec_data61.tsv
